In [ ]:
import os
from sys import stdout

import nglview as ngl
import pytraj as pt
import numpy as np
import matplotlib.pyplot as plt

# WTM-eABF for a particle in a double-well potential

### Setup the simulation

Here we will use the WTM-eABF sampling algorithm along the x-axis. 

In [ ]:
from adaptive_sampling.sampling_tools import *
from adaptive_sampling.interface.interfaceMD_2D import MD
from adaptive_sampling.units import *

# ------------------------------------------------------------------------------------
# define collective variables
cv_atoms        = []              # not needed for 2D potentials
minimum         = 70.0            # minimum of the CV
maximum         = 170.0           # maximum of the CV
bin_width       = 2.0             # bin with along the CV

collective_var = [["x", cv_atoms, minimum, maximum, bin_width]]

# ------------------------------------------------------------------------------------
# setup MD
mass      = 10.0   # mass of particle in a.u.
seed      = 42     # random seed
dt        = 5.0e0  # stepsize in fs
temp      = 300.0  # temperature in K

coords_in = [71.0, 0.5]

the_md = MD(
    mass_in=mass,
    coords_in=coords_in,
    potential="1",
    dt_in=dt,
    target_temp_in=temp,
    seed_in=seed,
)
the_md.calc_init()
the_md.calc_etvp()

# --------------------------------------------------------------------------------------
# Setup the sampling algorithm
eabf_ext_sigma    = 2.0     # thermal width of coupling between CV and extended variable 
eabf_ext_mass     = 20.0    # mass of extended variable in a.u.
abf_nfull         = 500     # number of samples per bin when abf force is fully applied
mtd_hill_height   = 1.0     # MtD hill height in kJ/mol   
mtd_hill_std      = 4.0     # MtD hill width
mtd_well_tempered = 1000.0  # MtD Well-tempered temperature
mtd_frequency     = 100     # MtD frequency of hill creation

the_bias = WTMeABF(
    eabf_ext_sigma, 
    eabf_ext_mass, 
    mtd_hill_height,
    mtd_hill_std,
    the_md, 
    collective_var,         # collective variable
    output_freq=1000,       # frequency of writing outputs
    f_conf=0.0,             # confinement force of CV at boundaries
    nfull=abf_nfull,        
    equil_temp=temp,        # equilibrium temperature of simulation
    well_tempered_temp=mtd_well_tempered,
    hill_drop_freq=mtd_frequency,
    force_from_grid=True,   # accumulate metadynamics force and bias on grid
    kinetics=True,          # calculate importent metrics to get accurate kinetics
    verbose=False,          # print verbose output
)
the_bias.step_bias()

In [ ]:
def print_output(the_md):
    print("%11.2f\t%14.6f\t%14.6f\t%14.6f\t%14.6f\t%14.6f\t%14.6f" % (
        the_md.step * the_md.dt * atomic_to_fs,
        the_md.coords[0],
        the_md.coords[1],
        the_md.epot,
        the_md.ekin,
        the_md.epot + the_md.ekin,
        the_md.temp,
    ))

### Run MD

In [ ]:
nsteps  = 100000
outfreq = 1000
x,y     = [],[]

print(
    "%11s\t%14s\t%14s\t%14s\t%14s\t%14s\t%14s"
    % ("time [fs]", "x", "y", "E_pot", "E_kin", "E_tot", "Temp")
)
print_output(the_md)

while the_md.step < nsteps:
    the_md.step += 1

    the_md.propagate(langevin=True)
    the_md.calc()

    the_md.forces += the_bias.step_bias()

    the_md.up_momenta(langevin=True)
    the_md.calc_etvp()

    if the_md.step % outfreq == 0:
        print_output(the_md)
        x.append(the_md.coords[0])
        y.append(the_md.coords[1])

In [ ]:
cv_traj = np.loadtxt('CV_traj.dat', skiprows=1)

In [ ]:
fig, axs = plt.subplots(1, 1, sharey=False, figsize=(8,6))
axs.scatter(cv_traj[::10,0]/1000, cv_traj[::10,1], s=1)
#axs.set_yticks([-180,0,180])
axs.set_xlabel('time / ps', fontsize=30)
axs.set_ylabel('CV / Degree', fontsize=30)
axs.tick_params(axis='y',length=6,width=3,labelsize=25, pad=10, direction='in')
axs.tick_params(axis='x',length=6,width=3,labelsize=25, pad=10, direction='in')
axs.spines['bottom'].set_linewidth(3)
axs.spines['top'].set_linewidth(3)
axs.spines['left'].set_linewidth(3)
axs.spines['right'].set_linewidth(3)
fig.tight_layout()

### Compute the PMF

Now we will use the MBAR estimator to calculate the unbiased weights of simulation frames. From those we compute the PMF along $x$.

In [ ]:
from adaptive_sampling.processing_tools import mbar
ext_sigma = 2.0    # thermal width of coupling between CV and extended variable 

# grid for free energy profile can be different than during sampling
minimum   = 70.0    
maximum   = 170.0    
bin_width = 1.0  
grid = np.arange(minimum, maximum, bin_width)

cv = cv_traj[:,1]  # trajectory of collective variable
la = cv_traj[:,2]  # trajectory of extended system

# run MBAR and compute free energy profile and probability density from statistical weights
traj_list, indices, meta_f = mbar.get_windows(grid, cv, la, ext_sigma, equil_temp=300.0)

exp_U, frames_per_traj = mbar.build_boltzmann(
    traj_list, 
    meta_f, 
    equil_temp=300.0,
)

weights = mbar.run_mbar(
    exp_U,
    frames_per_traj,
    max_iter=10000,
    conv=1.0e-4,
    conv_errvec=1.0,
    outfreq=100,
    device='cpu',
)

pmf_mbar, rho_mbar = mbar.pmf_from_weights(grid, cv[indices], weights, equil_temp=300.0)

In [ ]:
fig, axs = plt.subplots(1, 1, sharey=False, figsize=(8,6))

#plt.plot(np.degrees(the_bias.grid[0]), the_bias.pmf[0], linewidth=5)
plt.plot(grid, pmf_mbar-pmf_mbar.min(), linewidth=5)

axs.set_xlabel(r'$x$', fontsize=30)
axs.set_ylabel(r'$A(x)$', fontsize=30)
axs.set_xticks([80,120,160])

axs.tick_params(axis='y',length=6,width=3,labelsize=25, pad=10, direction='in')
axs.tick_params(axis='x',length=6,width=3,labelsize=25, pad=10, direction='in')
axs.spines['bottom'].set_linewidth(3)
axs.spines['top'].set_linewidth(3)
axs.spines['left'].set_linewidth(3)
axs.spines['right'].set_linewidth(3)
fig.tight_layout()

### Sampling of the $(x,y)$ plane 

In [ ]:
from matplotlib.patches import Rectangle
fig, axs = plt.subplots(1, 1, sharey=False, figsize=(8,6))

axs.scatter(x, y, alpha=0.5, s=20)

# formatting
axs.set_xlabel(r'$x$', fontsize=30)
axs.set_ylabel(r'$y$', fontsize=30)
axs.tick_params(axis='x',length=6,width=3,labelsize=25, pad=10, direction='in')
axs.tick_params(axis='y',length=6,width=3,labelsize=25, pad=10, direction='in')
axs.spines['bottom'].set_linewidth(3)
axs.spines['top'].set_linewidth(3)
axs.spines['left'].set_linewidth(3)
axs.spines['right'].set_linewidth(3)
fig.tight_layout()